In [1]:
import json

In [2]:
from tqdm import tqdm

In [3]:
bing_results = []
with open('../retrieval_importance/applications/imputation/bing/restaurant/test.jsonl', 'r') as bing_results_file:
    while (line := bing_results_file.readline()):
        bing_results.append(json.loads(line))

In [4]:
gpt_answers = []
with open('../retrieval_importance/applications/imputation/answers/restaurant_gptjt6b/__result__restaurant_test.txt', 'r') as gpt_file:
    while (line := gpt_file.readline()):    
        gpt_answers.append(line)
    

In [5]:
cleaned_answers = []

for answer in gpt_answers:
    cleaned_answer = answer
    cleaned_answer = cleaned_answer.strip()
    cleaned_answer = cleaned_answer.split('.')[0]
    cleaned_answer = cleaned_answer.split(',')[0]
    cleaned_answer = cleaned_answer.replace('"', '')
    cleaned_answer = cleaned_answer.lower()
    
    if len(cleaned_answer)> 0 and cleaned_answer[0] == ":":
        cleaned_answer = cleaned_answer[1:]
        cleaned_answer = cleaned_answer.strip()
    
    #print(cleaned_answer, f'[{answer.strip()}]')
    cleaned_answers.append(cleaned_answer)

In [6]:
answers = []

gpt_answer_index = 0

for sample_index in tqdm(range(len(bing_results))):
    pages = bing_results[sample_index]['webPages']['value']

    for position, page in enumerate(pages):

        answer = cleaned_answers[gpt_answer_index]

        answers.append((sample_index, position, answer, page['url']))
        gpt_answer_index += 1
    


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 86/86 [00:00<00:00, 10552.02it/s]


In [7]:
import pandas as pd
test_answers = pd.DataFrame.from_records(answers, columns=['sample_index', 'position', 'answer', 'url'])

test_answers.to_csv('../retrieval_importance/applications/imputation/answers/restaurant_gptjt6b/test.csv', index=False)
